In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
   
    name=t.text_content()
    print("%s"%(name))
    col.append((name,[]))

Postcode
Borough
Neighbourhood



In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)


In [9]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)


In [10]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [11]:
df.shape

(211, 3)

In [30]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
dFrame = pd.merge(df, coordinates, on=['Postcode'], how='inner')
dFrame.rename(columns={'Neighbourhood\n':'Neighbourhood'}, inplace=True)

In [64]:
dFrame.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods\n,43.753259,-79.329656
1,M4A,North York,Victoria Village\n,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront\n,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park\n,43.654260,-79.360636
4,M6A,North York,Lawrence Heights\n,43.718518,-79.464763


In [72]:
df = dFrame['Neighbourhood'].groupby([dFrame.Postcode, dFrame.Borough, dFrame.Latitude, dFrame.Longitude]).apply(list).reset_index()
df = df[['Postcode', 'Borough','Neighbourhood','Latitude','Longitude']]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge\n, Malvern\n]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek\n, Rouge Hill\n, Port Union\n]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood\n, Morningside\n, West Hill\n]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn\n],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae\n],43.773136,-79.239476
